In [19]:
import sys
import importlib
import numpy as np
sys.path.insert(0, '../src/')

In [20]:
import dataset
importlib.reload(dataset)

m = dataset.load_messenger_conversations('/home/patz/Heap/messages/')

In [5]:
importlib.reload(dataset)
m = [dataset.rename_conversation_user(c, 'Patztablook TwentyTwo', 'you') for c in m]

In [21]:
importlib.reload(dataset)

def predicate(content):
    if content is None: return True
    content = content.strip()
    blacklist = ['ə', 'ɥ', 'ʇ', 'ʎ', 'ɹ', 'ƃ',
                'se svymi geny? 0=ne',
                'are you satisfied with ur genes?',
                'would you describe urself as']
    for b in blacklist:
        if b in content: return False
    if content in ['.', '*'] : return False # probably a reference to a previous message
    # fixing previous a previous message (probably a typo):
    if content[0] == '*' and content[1] != ' ' and len(content.split(' ')) <= 2 and content.count('*') == 1: return False
    if content[-1] == '*' and content[-2] != ' ' and len(content.split(' ')) <= 2 and content.count('*') == 1: return False
    return True

m2 = [dataset.filter_messages_by_content(c, predicate) for c in m]

In [22]:
m2 = list(filter(lambda c: len(c) >= 64, m2))
print(len(m2))
for c in m:
    print(c.title, c.participants, len(c))

49
Scanthning Brot puɐ Vincent Fournier {'Scanthning Brot', 'Vincent Fournier', 'Patztablook TwentyTwo'} 7
Pendulum Swing {'Rawen Dorfin', 'Martin McNickle', 'Robin Stringer', 'Patztablook TwentyTwo', 'Sigismund Häggkvist'} 15750
Amélie Novotná {'Patztablook TwentyTwo', 'Amélie Novotná'} 156
Martin McNickle {'Patztablook TwentyTwo', 'Martin McNickle'} 6
Ondřej Smolík {'Patztablook TwentyTwo', 'Ondřej Smolík'} 176
Alexander Terziev {'Patztablook TwentyTwo', 'Alexander Terziev'} 4433
Manka Roblíková {'Patztablook TwentyTwo', 'Manka Roblíková'} 745
Matouš Pikous {'Matouš Pikous', 'Patztablook TwentyTwo'} 35
Jan Zasadil {'Jan Zasadil', 'Patztablook TwentyTwo'} 38947
Katie Alzbeta Brown {'Patztablook TwentyTwo', 'Katie Alzbeta Brown'} 102
Jan Vaniš {'Patztablook TwentyTwo', 'Jan Vaniš'} 407
Katie Alzbeta Brown puɐ Karel Loula {'Patztablook TwentyTwo', 'Katie Alzbeta Brown', 'Karel Loula'} 183
Filip Kastl {'Patztablook TwentyTwo', 'Filip Kastl'} 6845
The only magician capable of a craft left

In [23]:
val_indices = [] #[-1, -3, -13]
train_indices = np.delete(np.arange(len(m2)), val_indices)

print(len(m2))
for i in val_indices:
    print(m2[i].participants)

49


In [24]:
import dataset
importlib.reload(dataset)

special_tokens = {
    'conversation_start': '[CSTART]',
    'conversation_end': '[CEND]',
    'message_start': '[MSTART]',
    'breakpoint': '[BREAK]',
    'message_end': '[MEND]',
    'writes': '[WRITES]',
}

dataset.generate_corpus('../data/train.txt', [m2[i] for i in train_indices], special_tokens, break_tollerance_s=180)
dataset.generate_corpus('../data/val.txt', [m2[i] for i in val_indices], special_tokens, break_tollerance_s=180)